In [24]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [25]:
ex1 = pd.read_parquet('train/6.채널정보/2018_채널정보_전체.parquet')

In [26]:
member_df = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')
member_seg = member_df[['ID', 'Segment']].drop_duplicates(subset='ID')
ex1 = ex1.merge(member_seg, on='ID', how='left')



In [27]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M', '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문횟수_앱_R6M', '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문횟수_모바일웹_R6M', '방문일수_모바일웹_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '방문횟수_모바일웹_B0M', '방문일수_모바일웹_B0M', '인입횟수_IB_R6M', '인입횟수_금융_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M', '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M', '인입불만횟수_IB_R6M', '인입불만일수_IB_R6M', '인입불만월수_IB_R6M', '인입불만후경과월_IB_R6M', '인입불만횟수_IB_B0M', '인입불만일수_IB_B0M', '상담건수_B0M', 'IB문의건수_사용승인내역_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_선결제_B0M', 'IB문의건수_결제_B0M', 'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M', 'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M', 'IB문의건수_포인트_B0M', 'IB문의건수_카드발급_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M', 'IB문의건수_

In [28]:
cols = ['인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문횟수_앱_R6M', 'OS구분코드']
print(ex1[cols].head())

  인입횟수_ARS_R6M 이용메뉴건수_ARS_R6M 방문횟수_PC_R6M 방문일수_PC_R6M 방문횟수_앱_R6M   OS구분코드
0       10회 이상         10회 이상       1회 이상       1회 이상      1회 이상  Android
1        1회 이상          1회 이상       1회 이상       1회 이상      1회 이상     None
2        1회 이상          1회 이상      10회 이상      10회 이상     30회 이상  Android
3       10회 이상         10회 이상       1회 이상       1회 이상      1회 이상  Android
4        1회 이상          1회 이상       1회 이상       1회 이상      1회 이상  Android


In [29]:
count_cols2 = ['인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문횟수_앱_R6M']

for col in count_cols2:
    ex1[col] = ex1[col].astype(str).str.replace('회 이상', '', regex=False)
    
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [30]:
le_cols = ['OS구분코드']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [31]:
ex1.head()

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
0,201807,TRAIN_000000,10,10,8,6,0,2,6,2,...,0,22,221,6,0,0,0,0,0,D
1,201807,TRAIN_000001,1,1,0,0,0,0,0,0,...,0,0,0,0,-1,0,0,0,0,E
2,201807,TRAIN_000002,1,1,1,1,0,2,5,1,...,0,0,0,0,0,11,6,5,5,C
3,201807,TRAIN_000003,10,10,10,6,0,2,6,2,...,0,23,219,6,0,0,0,0,0,D
4,201807,TRAIN_000004,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [32]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '인입횟수_ARS_R6M', '이용메뉴건수_ARS_R6M', '인입일수_ARS_R6M', '인입월수_ARS_R6M', '인입후경과월_ARS', '인입횟수_ARS_B0M', '이용메뉴건수_ARS_B0M', '인입일수_ARS_B0M', '방문횟수_PC_R6M', '방문일수_PC_R6M', '방문월수_PC_R6M', '방문후경과월_PC_R6M', '방문횟수_앱_R6M', '방문일수_앱_R6M', '방문월수_앱_R6M', '방문후경과월_앱_R6M', '방문횟수_모바일웹_R6M', '방문일수_모바일웹_R6M', '방문월수_모바일웹_R6M', '방문후경과월_모바일웹_R6M', '방문횟수_PC_B0M', '방문일수_PC_B0M', '방문횟수_앱_B0M', '방문일수_앱_B0M', '방문횟수_모바일웹_B0M', '방문일수_모바일웹_B0M', '인입횟수_IB_R6M', '인입횟수_금융_IB_R6M', '인입일수_IB_R6M', '인입월수_IB_R6M', '이용메뉴건수_IB_R6M', '인입후경과월_IB_R6M', '인입횟수_IB_B0M', '인입일수_IB_B0M', '이용메뉴건수_IB_B0M', '인입불만횟수_IB_R6M', '인입불만일수_IB_R6M', '인입불만월수_IB_R6M', '인입불만후경과월_IB_R6M', '인입불만횟수_IB_B0M', '인입불만일수_IB_B0M', '상담건수_B0M', 'IB문의건수_사용승인내역_B0M', 'IB문의건수_한도_B0M', 'IB문의건수_선결제_B0M', 'IB문의건수_결제_B0M', 'IB문의건수_할부_B0M', 'IB문의건수_정보변경_B0M', 'IB문의건수_결제일변경_B0M', 'IB문의건수_명세서_B0M', 'IB문의건수_비밀번호_B0M', 'IB문의건수_SMS_B0M', 'IB문의건수_APP_B0M', 'IB문의건수_부대서비스_B0M', 'IB문의건수_포인트_B0M', 'IB문의건수_카드발급_B0M', 'IB문의건수_BL_B0M', 'IB문의건수_분실도난_B0M', 'IB문의건수_

In [33]:
ex1 = ex1.fillna(-1)
ex1

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,당사PAY_방문월수_R6M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,OS구분코드,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M,Segment
0,201807,TRAIN_000000,10,10,8,6,0,2,6,2,...,0,22,221,6,0,0,0,0,0,D
1,201807,TRAIN_000001,1,1,0,0,0,0,0,0,...,0,0,0,0,-1,0,0,0,0,E
2,201807,TRAIN_000002,1,1,1,1,0,2,5,1,...,0,0,0,0,0,11,6,5,5,C
3,201807,TRAIN_000003,10,10,10,6,0,2,6,2,...,0,23,219,6,0,0,0,0,0,D
4,201807,TRAIN_000004,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,1,1,0,0,0,0,0,0,...,0,0,0,0,-1,0,0,0,0,E
2399996,201812,TRAIN_399996,1,1,0,0,0,0,0,0,...,0,0,0,1,-1,0,0,0,0,D
2399997,201812,TRAIN_399997,1,1,0,0,0,0,0,0,...,0,0,0,0,-1,0,0,0,0,C
2399998,201812,TRAIN_399998,1,1,0,0,0,0,0,0,...,0,0,0,0,-1,0,0,0,0,E


In [34]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['방문횟수_모바일웹_B0M',
 '방문일수_모바일웹_B0M',
 '인입횟수_금융_IB_R6M',
 '인입불만횟수_IB_R6M',
 '인입불만일수_IB_R6M',
 '인입불만월수_IB_R6M',
 '인입불만후경과월_IB_R6M',
 '인입불만횟수_IB_B0M',
 '인입불만일수_IB_B0M',
 'IB문의건수_사용승인내역_B0M',
 'IB문의건수_한도_B0M',
 'IB문의건수_선결제_B0M',
 'IB문의건수_결제_B0M',
 'IB문의건수_할부_B0M',
 'IB문의건수_정보변경_B0M',
 'IB문의건수_결제일변경_B0M',
 'IB문의건수_명세서_B0M',
 'IB문의건수_비밀번호_B0M',
 'IB문의건수_SMS_B0M',
 'IB문의건수_APP_B0M',
 'IB문의건수_부대서비스_B0M',
 'IB문의건수_포인트_B0M',
 'IB문의건수_카드발급_B0M',
 'IB문의건수_BL_B0M',
 'IB문의건수_분실도난_B0M',
 'IB문의건수_CA_B0M',
 'IB문의건수_CL_RV_B0M',
 'IB문의건수_CS_B0M',
 'IB상담건수_VOC_B0M',
 'IB상담건수_VOC민원_B0M',
 'IB상담건수_VOC불만_B0M',
 'IB상담건수_금감원_B0M',
 'IB문의건수_결제_R6M',
 'IB문의건수_할부_R6M',
 'IB문의건수_명세서_R6M',
 'IB문의건수_비밀번호_R6M',
 'IB문의건수_SMS_R6M',
 'IB문의건수_APP_R6M',
 'IB문의건수_부대서비스_R6M',
 'IB문의건수_포인트_R6M',
 'IB문의건수_BL_R6M',
 'IB문의건수_분실도난_R6M',
 'IB문의건수_CA_R6M',
 'IB상담건수_VOC_R6M',
 'IB상담건수_VOC민원_R6M',
 'IB상담건수_VOC불만_R6M',
 'IB상담건수_금감원_R6M',
 '불만제기건수_B0M',
 '불만제기건수_R12M',
 '당사PAY_방문횟수_B0M',
 '당사PAY_방문횟수_R6M',
 '당사PAY_방문월수_R6M']

In [35]:
cols_to_drop = ['방문횟수_모바일웹_B0M',
 '방문일수_모바일웹_B0M',
 '인입횟수_금융_IB_R6M',
 '인입불만횟수_IB_R6M',
 '인입불만일수_IB_R6M',
 '인입불만월수_IB_R6M',
 '인입불만후경과월_IB_R6M',
 '인입불만횟수_IB_B0M',
 '인입불만일수_IB_B0M',
 'IB문의건수_사용승인내역_B0M',
 'IB문의건수_한도_B0M',
 'IB문의건수_선결제_B0M',
 'IB문의건수_결제_B0M',
 'IB문의건수_할부_B0M',
 'IB문의건수_정보변경_B0M',
 'IB문의건수_결제일변경_B0M',
 'IB문의건수_명세서_B0M',
 'IB문의건수_비밀번호_B0M',
 'IB문의건수_SMS_B0M',
 'IB문의건수_APP_B0M',
 'IB문의건수_부대서비스_B0M',
 'IB문의건수_포인트_B0M',
 'IB문의건수_카드발급_B0M',
 'IB문의건수_BL_B0M',
 'IB문의건수_분실도난_B0M',
 'IB문의건수_CA_B0M',
 'IB문의건수_CL_RV_B0M',
 'IB문의건수_CS_B0M',
 'IB상담건수_VOC_B0M',
 'IB상담건수_VOC민원_B0M',
 'IB상담건수_VOC불만_B0M',
 'IB상담건수_금감원_B0M',
 'IB문의건수_결제_R6M',
 'IB문의건수_할부_R6M',
 'IB문의건수_명세서_R6M',
 'IB문의건수_비밀번호_R6M',
 'IB문의건수_SMS_R6M',
 'IB문의건수_APP_R6M',
 'IB문의건수_부대서비스_R6M',
 'IB문의건수_포인트_R6M',
 'IB문의건수_BL_R6M',
 'IB문의건수_분실도난_R6M',
 'IB문의건수_CA_R6M',
 'IB상담건수_VOC_R6M',
 'IB상담건수_VOC민원_R6M',
 'IB상담건수_VOC불만_R6M',
 'IB상담건수_금감원_R6M',
 '불만제기건수_B0M',
 '불만제기건수_R12M',
 '당사PAY_방문횟수_B0M',
 '당사PAY_방문횟수_R6M',
 '당사PAY_방문월수_R6M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [36]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['방문일수_PC_B0M', '방문일수_앱_B0M', '인입횟수_ARS_R6M', '인입횟수_IB_B0M', 'IB문의건수_CS_R6M', 'IB문의건수_한도_R6M', '홈페이지_선결제건수_R6M', '홈페이지_선결제건수_R3M', '방문횟수_PC_B0M', '방문횟수_앱_B0M', 'IB문의건수_카드발급_R6M', 'IB문의건수_CL_RV_R6M', '당사멤버쉽_방문횟수_R6M', '방문후경과월_모바일웹_R6M', 'OS구분코드', '이용메뉴건수_IB_B0M', '방문월수_모바일웹_R6M', '방문월수_PC_R6M', '인입횟수_ARS_B0M', '방문횟수_PC_R6M', 'IB문의건수_선결제_R6M', '당사멤버쉽_방문횟수_B0M', '이용메뉴건수_ARS_R6M', '당사멤버쉽_방문월수_R6M', '방문횟수_모바일웹_R6M', '홈페이지_금융건수_R3M', '인입일수_ARS_B0M', '방문일수_모바일웹_R6M', '방문후경과월_PC_R6M', '방문횟수_앱_R6M', 'IB문의건수_사용승인내역_R6M', 'IB문의건수_정보변경_R6M', '이용메뉴건수_ARS_B0M', '홈페이지_금융건수_R6M', 'IB문의건수_결제일변경_R6M', '인입일수_IB_B0M', '방문일수_PC_R6M', '상담건수_B0M']


In [37]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)

high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 29


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,인입횟수_IB_R6M,인입일수_IB_R6M,0.973256,0.140527,0.142005
1,인입횟수_IB_R6M,상담건수_R6M,0.968979,0.140527,0.143577
2,인입일수_IB_R6M,이용메뉴건수_IB_R6M,0.963851,0.142005,0.134681
3,인입횟수_IB_R6M,이용메뉴건수_IB_R6M,0.953396,0.140527,0.134681
4,인입일수_IB_R6M,상담건수_R6M,0.944073,0.142005,0.143577
5,방문월수_앱_R6M,방문후경과월_앱_R6M,0.943714,0.152753,0.158202
6,이용메뉴건수_IB_R6M,상담건수_R6M,0.923367,0.134681,0.143577
7,인입일수_ARS_R6M,인입월수_ARS_R6M,0.916620,0.156965,0.154813
8,인입월수_IB_R6M,상담건수_R6M,0.904594,0.142631,0.143577
9,인입횟수_IB_R6M,인입월수_IB_R6M,0.900078,0.140527,0.142631


In [38]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))

print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 9
제거할 피처 목록:
['방문월수_앱_R6M', '인입월수_IB_R6M', '방문일수_앱_R6M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입후경과월_IB_R6M', '인입월수_ARS_R6M', '이용메뉴건수_IB_R6M', '상담건수_R6M']


In [39]:
cols_to_drop = ['방문월수_앱_R6M', '인입월수_IB_R6M', '방문일수_앱_R6M', '인입횟수_IB_R6M', '인입일수_IB_R6M', '인입후경과월_IB_R6M', '인입월수_ARS_R6M', '이용메뉴건수_IB_R6M', '상담건수_R6M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [40]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [41]:
ex1

,기준년월,ID,인입일수_ARS_R6M,인입후경과월_ARS,방문후경과월_앱_R6M,불만제기후경과월_R12M,Segment,Segment_encoded
0,201807,TRAIN_000000,8,0,6,12,D,3
1,201807,TRAIN_000001,0,0,6,12,E,4
2,201807,TRAIN_000002,1,0,0,12,C,2
3,201807,TRAIN_000003,10,0,6,12,D,3
4,201807,TRAIN_000004,0,0,6,0,E,4
...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,6,0,E,4
2399996,201812,TRAIN_399996,0,0,6,12,D,3
2399997,201812,TRAIN_399997,0,0,6,12,C,2
2399998,201812,TRAIN_399998,0,0,6,0,E,4


In [42]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [43]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월', 'ID', 'Segment', '인입일수_ARS_R6M', '방문후경과월_앱_R6M', '불만제기후경과월_R12M']

In [44]:
ex1_filtered.to_parquet('채널_전처리_Segment.parquet', index=False)